## Web Scraping and Writing to Google Sheets
    1. Get the data and parse with requests, lxml and Beatiful Soup 
    2. Data wrange into dictionaries 
    3. Create Pandas DF and clean the data
    4. Write to Google Sheets 

In [ ]:
from requests import get 
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup 

In [ ]:
#imputing URL and creating header creds for parsing 
url = 'http://www.nfl.com/stats/weeklyleaders?week=15&season=2019&showCategory=Passing'
my_header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36'}


In [ ]:
#grabbing the HTML and getting text 
fantasy_page = get(url, headers = my_header)

doc = lh.fromstring(fantasy_page.content)

In [ ]:
print(fantasy_page)

Response == 200 means the page scraped successfully 

In [ ]:
#parsing table elements in the HTML inside the pattern "//tr" --> this is a table element 

table_elements = doc.xpath('//tr')

In [ ]:
#[len(T) for T in table_elements[:-1]]

In [ ]:
title = doc.xpath('//tr//th')

In [ ]:
colnames = []

n = len(title)

for i in range(0, n):
    name = title[i].text_content()
    colnames.append(name)
    #print(i) 


In [ ]:
colnames

In [ ]:
#creating an empty array to insert the table elements 
cols = []

i = 0 #setting the increment 

for j in range(0, len(colnames)):
    i+1
    name = colnames[j] #getting the column name from the HTML table 
    #print('%d:"%s"'% (i, name))
    cols.append((name, []))
    

In [ ]:
#Since out first row is the header, data is stored on the second row onwards

for j in range(1,len(table_elements)):
    #T is our j'th row
    T=table_elements[j]
    
    #If row is not of size 24, the //tr data is not from our table 
    if len(T)!=12:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 

        #Append the data to the empty list of the i'th column
        cols[i][1].append(data)
        #Increment i for the next column
        i+=1

In [ ]:
#creating a dictionary for the columns in the parsed table 
Dict={title:column for (title,column) in cols}

df=pd.DataFrame(Dict)

In [ ]:
df

### Light data cleaning 
    1. Dropping the "Rk" row because it is empty 
    2. Dropping a blank column used for formatting in the HTML 

In [ ]:
escapes = ''.join([chr(char) for char in range(1, 32)])

translator = str.maketrans('', '', escapes)

In [ ]:
df.columns = df.columns.str.translate(translator)

In [ ]:
fixed = ['Name', 'Team', 'Opp', 'Score']

for i in fixed:
    df.loc[:, i] = df.loc[:, i].str.translate(translator)

In [ ]:
df

#### Data Validation with Pydantic 

In [ ]:
df.columns

In [ ]:
from pydantic import BaseModel

class scrapped_passing(BaseModel):
    Name: str
    Team: str
    Opp: str 
    Score: str 
    Comp: int 
    Att: int 
    Yds: float
    TD: int 
    Int: int
    Sck: int
    FUM: int
    Rate: float

class scrapped_recieving(BaseModel):
    Name: str
    Team: str
    Opp: str 
    Score: str
    Rec: int 
    Yds: float
    Avg: float 
    TD: int
    Fum: int 
        
class scrapped_rushing(BaseModel):
    Name: str
    Team: str
    Opp: str 
    Score: str
    Rec: int 
    Yds: float
    Avg: float 
    TD: int
    Fum: int 

In [ ]:
scraped_passing(Name = df.loc[:, 'Name'],
               Team = df.loc[:, 'Team'],
               Opp = df.loc[:, 'Opp'],
               Score = df.loc[:, 'Score'],
               Comp = df.loc[:, 'Comp'],
               Att = df.loc[:, 'Att'],
               Tds = df.loc[:, 'Yds'],
               TD = df.loc[:, 'TD'],
               Int = df.loc[:, 'Int'],
               Sck = df.loc[:, 'Sck'],
               FUM = df.loc[:, 'FUM'],
               Rate = df.loc[:, 'Rate'])

In [ ]:
df.dtypes

In [ ]:
#Grapping Parameters for looping 
n_rows = df.shape[0]
n_cols = df.shape[1]

In [ ]:
#import library
import gspread 
#Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Print nicely
import pprint
#Create scope
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
#create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name('quickstart/g_sheet_creds.json',scope)
#create gspread authorize using that credential
client = gspread.authorize(creds)

In [ ]:
# creating a new sheet to drop the data 
#new_sheet = client.create('target_sheet')

#new_sheet.share('players@fantasy-data-build.iam.gserviceaccount.com', perm_type='user', role='writer')

In [ ]:
#sheet = client.open('target_sheet')

#ws = sheet.get_worksheet(0)

In [ ]:
#Now will can access our google sheets we call client.open on StartupName
test_url = 'ttps://docs.google.com/spreadsheets/d/1253330Gq5L7fnSb9e7rz0ka7fjXGfe4F9I2AzUkkz3w/edit?folder=0AHsp7iQa2x1DUk9PVA#gid=0'
sheet = client.open_by_url(test_url)

ws = sheet.get_worksheet(0)

#Access all of the record inside that
#result = sheet.get_all_record()

shaped_data = df.transpose()

ws.insert_row(df.columns.tolist(), 1)

for i in range(1, n_rows): 
    row = df.iloc[i-1].tolist()
    index = i+1
    ws.insert_row(row, index)
    print(i)
